SPDX-FileCopyrightText: SAS research group, HFT, Helmut Schmidt University

SPDX-License-Identifier: CC0-1.0

https://github.com/hsu-sonar/icua24-geopackage

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import widgets

%matplotlib ipympl

# Calculating the line piece distances

The change detection algorithm makes use of the *line piece average distance* metric, a measure of the distance between two line segments. The steps to find these distances is given in the appendix of the paper. In this notebook, an implementation of these calculations is developed. This corresponds to the `line_piece_distances()` function in the `metrics` library module.

## Intermediate distances

The line piece distances are calculated from four intermediate distances, $d1, d_2, d_3$ and $d_4$. As shown in the figure below (corresponding to Figure 6a in the paper), these are the distances from the corresponding endpoints on one line - $P_1, P_2, P_3$ and $P_4$, respectively - to a perpendicular point on the other line (or, if no perpendicular point exists on the other line, then the distance to the endpoint on the other line closest to perpendicular).

<div style="text-align: center;"></div>

To make the mathematics clearer, let $P$ be the endpoint of interest, and call the endpoints of the other line segment $A$ and $B$ and the point we are trying to find $H$. For example, if we were calculating $d_2$, then we would have the following:
* $P = P_2$
* $A = P_1$
* $B = P_3$

We know $H$ lies somewhere on the line between $A$ and $B$. We can represent this with the parametric equation

\begin{equation}
H = A + t(B - A),
\end{equation}

where $t$ is the distance of the point from $A$ towards $B$ as a fraction of the line segment length $||B - A||$. If possible, we want the line $PH$ to be perpendicular to $AB$; from the definition of the dot product, this gives us

\begin{equation}
(B - A)\cdot(P - H) = 0.
\end{equation}

Substituting the first equation into the second yields

\begin{equation}
(B - A) \cdot (P - A - t(B - A)) = 0.
\end{equation}

As the dot product is a bilinear form, this can be rearranged as

\begin{equation}
(B - A) \cdot (P - A) + (B - A)\cdot(-t(B - A)) = 0,
\end{equation}

and since $t$ is scalar, this becomes

\begin{equation}
\begin{aligned}
(B - A) \cdot (P - A) - t(B - A)\cdot(B - A) &= 0, \\
(B - A) \cdot (P - A) - t||B - A||^2 &= 0.
\end{aligned}
\end{equation}

Rearranging for $t$ gives us

\begin{equation}
t = \frac{(B - A)\cdot(P - A)}{||B - A||^2}.
\end{equation}

Note that $0 \leq t \leq 1$ means the perpendicular point lies on the line segment; other values means it lies on the extension of the segment to an infinite line. Clipping the calculated value so $t\in[0, 1]$ results in $H$ being the closest endpoint to perpendicular in this case. The intermediate distance is then trivial to calculate as $d = || H - P ||$.

The following simple function calculates the position of $H$:

In [ ]:
def find_intermediate_point(P, A, B):
    """Find the intermediate point for distance calculations.

    Parameters
    ----------
    P : array-like
        A two-element array containing the easting and northing position of the distance
        starting point.
    A, B : array-like
        Two-element arrays containing the easting and northing coordinates of the
        endpoints of the line.

    Returns
    -------
    H : numpy.ndarray
        A two-element array containing the easting and northing position of the
        intermediate point to use for the distance calculation.

    """
    # Ensure they are arrays.
    P = np.array(P)
    A = np.array(A)
    B = np.array(B)

    # Calculate the t value for an infinite line.
    t = np.dot(B - A, P - A) / np.linalg.norm(B - A) ** 2

    # Clip to ensure it is on the line segment.
    t = np.clip(t, 0, 1)

    # And calculate the point.
    return A + t * (B - A)

In [ ]:
find_intermediate_point([0, 0], [-10, 10], [30, 15])

The following cell generates an interactive plot allowing you to change the coordinates of the point and line and see the resulting position of $H$ and corresponding distance $d$.

In [ ]:
# Generate a figure with axes for the widgets and resulting plot.
dist_fig, dist_axes = plt.subplot_mosaic(
    [
        ["ptitle", "atitle", "btitle"],
        ["pe", "ae", "be"],
        ["pn", "an", "bn"],
        ["results", "results", "results"],
    ],
    figsize=(10, 7),
    height_ratios=[1, 1, 1, 15],
)

# Create the control widgets.
dist_axes["ptitle"].axis("off")
dist_axes["ptitle"].set_title("P")
dist_pe = widgets.TextBox(dist_axes["pe"], "E", initial=10)
dist_pn = widgets.TextBox(dist_axes["pn"], "N", initial=30)
dist_axes["atitle"].axis("off")
dist_axes["atitle"].set_title("A")
dist_ae = widgets.TextBox(dist_axes["ae"], "E", initial=0)
dist_an = widgets.TextBox(dist_axes["an"], "N", initial=0)
dist_axes["btitle"].axis("off")
dist_axes["btitle"].set_title("B")
dist_be = widgets.TextBox(dist_axes["be"], "E", initial=100)
dist_bn = widgets.TextBox(dist_axes["bn"], "N", initial=8)


# Function to update the plot when the widgets are changed.
def dist_update(*args):
    # Load the widget values into arrays.
    P = np.array([float(dist_pe.text), float(dist_pn.text)])
    A = np.array([float(dist_ae.text), float(dist_an.text)])
    B = np.array([float(dist_be.text), float(dist_bn.text)])

    # Call our function.
    H = find_intermediate_point(P, A, B)

    # Resulting distance.
    D = P - H
    d = np.linalg.norm(D)

    # Angle between the lines.
    V = B - A
    angle = np.degrees(np.arccos(np.dot(D, V) / (d * np.linalg.norm(V))))

    # Coordinates to write the results at.
    tp = (P + H) / 2
    tp[0] += d / 50

    # Plot everything.
    ax = dist_axes["results"]
    ax.cla()
    ax.plot(*P, "C1o", label="Point P")
    ax.plot([A[0], B[0]], [A[1], B[1]], "C0o-", label="Line AB")
    ax.plot(*H, "rx", label="Intermediate point H")
    ax.plot([P[0], H[0]], [P[1], H[1]], "r--", label="Intermediate distance d")

    # Annotate with the distance and angle.
    ax.text(tp[0], tp[1], f"d: {d:.2f}", ha="left", va="bottom")
    ax.text(tp[0], tp[1], f"Angle: {angle:.2f}", ha="left", va="top")

    # And format it.
    ax.legend()
    ax.set_aspect("equal")
    ax.set_xlabel("Easting")
    ax.set_ylabel("Northing")


# Connect the update function to the widgets.
dist_pn.on_text_change(dist_update)
dist_pe.on_text_change(dist_update)
dist_an.on_text_change(dist_update)
dist_ae.on_text_change(dist_update)
dist_bn.on_text_change(dist_update)
dist_be.on_text_change(dist_update)
dist_update()

## Testing for an intersection

To calculate the line piece distances, we also need to determine if the two line segments intersect. Call the two line segments we are testing $AB$ and $CD$, and denote the potential intersection point $Q$. In the same manner as above, this intersection point can be parametrised in terms of its position along the two segments:

\begin{equation}
\begin{aligned}
Q &= A + (B - A)r, \\
Q &= C + (D - C)s,
\end{aligned}
\end{equation}

for parameters $r$ and $s$. If there is an intersection, these two parametrisations will be equal, i.e.,

\begin{equation}
\begin{aligned}
A + (B - A)r &= C + (D - C)s,\\
(B - A)r - (D - C)s &= C - A,\\
(B - A)r + (C - D)s &= C - A.
\end{aligned}
\end{equation}

We can separate the endpoints into their easting and northing components, and put the resulting system of equations into matrix form:

\begin{equation}
\begin{aligned}
\mathbf{Ax} &= \mathbf{b}, \\
\begin{bmatrix}B_e - A_e & C_e - D_e \\ B_n - A_n & C_n - D_n \end{bmatrix}
\begin{bmatrix}r \\ s\end{bmatrix}
&=
\begin{bmatrix}C_e - A_e \\ C_n - A_n\end{bmatrix}.
\end{aligned}
\end{equation}

We need to solve this for the $r$ and $s$ values of the potential intersection point. [Cramer's rule](https://en.wikipedia.org/wiki/Cramer%27s_rule) says the solution for the $i^\text{th}$ variable is

\begin{equation}
x_i = \frac{\det\mathbf{A}_i}{\det\mathbf{A}},
\end{equation}

where $\mathbf{A}_i$ is the matrix formed by replacing the $i^\text{th}$ column of $\mathbf{A}$ with $\mathbf{b}$. We can easily calculate the three required determinants:

\begin{equation}
\begin{aligned}
\det\mathbf{A} &= (B_e-A_e)(C_n-D_n) - (C_e-D_e)(B_n-A_n), \\
\det\mathbf{A}_r &= (C_e-A_e)(C_n-D_n) - (C_e-D_e)(C_n-A_n), \\
\det\mathbf{A}_s &= (B_e-A_e)(C_n-A_n) - (C_e-A_e)(B_n-A_n).
\end{aligned}
\end{equation}

These can be interpreted as a relationship between the gradients of line segments formed from different points. For example, rearranging the first equation we get

\begin{equation}
\frac{D_n-C_n}{D_e-C_e} = \frac{\det\mathbf{A}}{(B_e-A_e)(D_e-C_e)} + \frac{B_n-A_n}{B_e-A_e}.
\end{equation}

$\det\mathbf{A} = 0$ means the slope of $AB$ is the same as the slope of $CD$, $\det\mathbf{A}_r = 0$ means the slope of $AC$ is the same as the slope of $CD$, and $\det\mathbf{A}_s = 0$ means the slope of $AB$ is the same as the slope of $AC$.

We can divide the determinant values into three cases:

* $\det\mathbf{A}$ is zero and the others are not: the segments are parallel. Our intersection test must return false.
* All three determinants are zero: the segments are colinear. An intersection is possible if they overlap. However, in this case at least one of $d_1$ and $d_2$, and at least one of $d_3$ and $d_4$ will be zero. Hence both the minimum (equation 2) and average (equation 3) line piece distances will be zero and the intersection adjustment will have no affect. For simplicity, our intersection test can return false in this case although this would be incorrect for a general intersection test.
* All other cases: the infinite lines through the points intersect.

In the latter case, we do not care about the actual intersection point, just whether it occurs within the segments. For the first parameter, the intersection point would occur within its line segment if $0 \leq r \leq 1$, that is,

\begin{equation}
0 \leq \frac{\det\mathbf{A}_r}{\det\mathbf{A}} \leq 1.
\end{equation}

To avoid the division, we would like to adjust the equality test. We can expand and rearrange it and include the parallel line case to make our intersection test

\begin{equation}
\text{intersection within AB} =
\begin{cases}
0 \leq \det\mathbf{A}_r \leq \det\mathbf{A} & \text{if }\det\mathbf{A} > 0, \\
\text{false}                                & \text{if }\det\mathbf{A} = 0, \\
0 \geq \det\mathbf{A}_r \geq \det\mathbf{A} & \text{if }\det\mathbf{A} < 0.
\end{cases}
\end{equation}

If we multiply $\det\mathbf{A}_r$ by the sign of $\det\mathbf{A}$ and compare to the absolute value of $\det\mathbf{A}$, we can reduce this to two cases:

\begin{equation}
\text{intersection within AB} =
\begin{cases}
\text{false} & \text{if }\det\mathbf{A} = 0, \\
0 \leq \operatorname{sgn}(\det{\mathbf{A}}) \det\mathbf{A}_r \leq |\det\mathbf{A}| & \text{otherwise}.
\end{cases}
\end{equation}

The same procedure can be carried out for the $s$ parameter. Combining these gives the intersection test as

\begin{equation}
\text{segments intersect} = (\det\mathbf{A} \neq 0)
                            \land (0 \leq \operatorname{sgn}(\det{\mathbf{A}}) \det\mathbf{A}_r \leq |\det\mathbf{A}|)
                            \land (0 \leq \operatorname{sgn}(\det{\mathbf{A}}) \det\mathbf{A}_s \leq |\det\mathbf{A}|).
\end{equation}

The following simple function implements this test.

In [ ]:
def segments_intersect(A, B, C, D):
    """Check if two line segments intersect.

    Note that this will always return False for colinear lines as our use case does not
    require checking for overlapping segments. This is not valid for a general
    intersection test.

    Parameters
    ----------
    A, B, C, D : array-like
        Two-element arrays containing the easting and northing coordinates of the
        endpoints of the lines.

    Returns
    -------
    intersects : Boolean

    """
    # Calculate the three determinants.
    detA = (B[0] - A[0]) * (C[1] - D[1]) - (C[0] - D[0]) * (B[1] - A[1])
    detAr = (C[0] - A[0]) * (C[1] - D[1]) - (C[0] - D[0]) * (C[1] - A[1])
    detAs = (B[0] - A[0]) * (C[1] - A[1]) - (C[0] - A[0]) * (B[1] - A[1])

    # Multiply the per-variable determinants with the sign of the common and take the
    # absolute value of the common to simplify the comparison.
    sgn = np.sign(detA)
    detAr *= sgn
    detAs *= sgn
    detA = np.abs(detA)

    # Perform the test.
    return ~np.isclose(detA, 0) & (0 <= detAr <= detA) & (0 <= detAs <= detA)

In [ ]:
segments_intersect([0, 0], [100, 0], [10, -5], [10, 100])

The following cell generates an interactive plot allowing you to change the coordinates of the lines and see if an intersection occurs.

In [ ]:
# Generate a figure with axes for the widgets and resulting plot.
int_fig, int_axes = plt.subplot_mosaic(
    [
        ["atitle", "btitle", "ctitle", "dtitle"],
        ["ae", "be", "ce", "de"],
        ["an", "bn", "cn", "dn"],
        ["results", "results", "results", "results"],
    ],
    figsize=(10, 7),
    height_ratios=[1, 1, 1, 15],
)

# Create the control widgets.
int_axes["atitle"].axis("off")
int_axes["atitle"].set_title("A")
int_ae = widgets.TextBox(int_axes["ae"], "E", initial=0)
int_an = widgets.TextBox(int_axes["an"], "N", initial=0)
int_axes["btitle"].axis("off")
int_axes["btitle"].set_title("B")
int_be = widgets.TextBox(int_axes["be"], "E", initial=100)
int_bn = widgets.TextBox(int_axes["bn"], "N", initial=8)
int_axes["ctitle"].axis("off")
int_axes["ctitle"].set_title("C")
int_ce = widgets.TextBox(int_axes["ce"], "E", initial=0)
int_cn = widgets.TextBox(int_axes["cn"], "N", initial=-15)
int_axes["dtitle"].axis("off")
int_axes["dtitle"].set_title("D")
int_de = widgets.TextBox(int_axes["de"], "E", initial=51)
int_dn = widgets.TextBox(int_axes["dn"], "N", initial=3.5)


# Function to update the plot when the widgets are changed.
def int_update(*args):
    # Load the widget values into arrays.
    A = np.array([float(int_ae.text), float(int_an.text)])
    B = np.array([float(int_be.text), float(int_bn.text)])
    C = np.array([float(int_ce.text), float(int_cn.text)])
    D = np.array([float(int_de.text), float(int_dn.text)])

    # Call our function.
    intersects = segments_intersect(A, B, C, D)

    # Plot everything.
    ax = int_axes["results"]
    ax.cla()
    ax.plot([A[0], B[0]], [A[1], B[1]], label="AB")
    ax.plot([C[0], D[0]], [C[1], D[1]], label="CD")

    # Annotate with the result.
    common = dict(transform=ax.transAxes, ha="right", x=0.99, y=0.01, fontweight="bold")
    if intersects:
        ax.text(s="Intersects", color="r", **common)
    else:
        ax.text(s="Does not intersect", **common)

    # And format it.
    ax.legend()
    ax.set_aspect("equal")
    ax.set_xlabel("Easting")
    ax.set_ylabel("Northing")


# Connect the update function to the widgets.
int_ae.on_text_change(int_update)
int_an.on_text_change(int_update)
int_be.on_text_change(int_update)
int_bn.on_text_change(int_update)
int_ce.on_text_change(int_update)
int_cn.on_text_change(int_update)
int_de.on_text_change(int_update)
int_dn.on_text_change(int_update)
int_update()

## Complete calculation

The following function computes the line piece distances using the techniques described above. The assignment of the leg vertices to $P_1, P_2, P_3, P_4$ are as described in the paper. It can calculate the distances for multiple partner legs at the same time, using NumPy operations to avoid looping where possible.

In [ ]:
def line_piece_distances(base, partners, return_all=False):
    """Calculate line piece distances between a base segment and some partner segments.

    Note that the shape of the output arrays is described relative to the shape of the
    ``partners`` input array.

    Parameters
    ----------
    base : array-like
        An array [[point0_easting, point0_northing], [point1_easting, point1_northing]]
        specifying the endpoints of the base segment.
    partners : array-like
        An array containing the endpoints of the partner segments. This must be at least
        two dimensional with a shape (..., 2, 2). The last two dimensions contains the
        points in the same order as ``base``.
    return_all : Boolean
        If False, only the line piece minimum and line piece average distances will be
        returned. If True, the distances ``d1, ..., d4`` and the result of the
        intersection test will also be returned.

    Returns
    -------
    line_piece_minimum, line_piece_average : numpy.ndarray
        Arrays of the distances. These will have a shape (...).
    distances : numpy.ndarray
        The distances d1 through d4 used in the calculation. This will have a shape
        (4, ...) with the first axis corresponding to the different distances. It will
        only be returned if ``return_all`` is True.
    intersects : numpy.ndarray
        A Boolean array indicating which of the partner legs intersect the base leg.
        This will have a shape (...). It will only be returned if ``return_all`` is
        True.

    """
    # Ensure it is an array of the correct shape.
    base = np.array(base)
    if base.shape != (2, 2):
        raise ValueError("invalid shape for base coordinates")

    # Ensure it is an array with the last two dimensions being the correct shape.
    partners = np.array(partners)
    if partners.ndim < 2:
        raise ValueError("partner coordinate array must be at least 2d")
    if partners.shape[-2:] != (2, 2):
        raise ValueError("invalid shape for last 2 dimensions of partner coordinates")

    # Reserve space for the four sets of points.
    Pshape = [4] + list(partners.shape)
    del Pshape[-2]
    P = np.empty(Pshape, dtype=float)

    # Take the start and end of the base leg as P1 and P3.
    P[0] = base[0]
    P[2] = base[1]

    # Euclidean distances from P1 to start of partners and P3 to end of partners, and
    # the element-wise minimum (the metric u in the paper).
    distnorm_a = np.linalg.norm(partners[..., 0, :] - P[0], axis=-1)
    distnorm_b = np.linalg.norm(partners[..., 1, :] - P[2], axis=-1)
    distnorm = np.minimum(distnorm_a, distnorm_b)

    # Repeat for the flipped case.
    distflip_a = np.linalg.norm(partners[..., 1, :] - P[0], axis=-1)
    distflip_b = np.linalg.norm(partners[..., 0, :] - P[2], axis=-1)
    distflip = np.minimum(distflip_a, distflip_b)

    # Boolean mask of partners, True where P2 and P4 should be the start and finish of
    # the partner leg (the 'normal' case above).
    mask = distnorm <= distflip

    # Use this to select appropriate points for P2 and P4. The first argument to
    # select() is a list of masks and the second is a list of arrays. The output has
    # elements from array0 where mask0 is True and so on.
    mask = mask[..., np.newaxis]
    P[1] = np.select([mask, ~mask], [partners[..., 0, :], partners[..., 1, :]])
    P[3] = np.select([~mask, mask], [partners[..., 0, :], partners[..., 1, :]])

    # Allocate space for the intermediate distances.
    dshape = list(P.shape)[1:]
    dshape[-1] = 4
    d = np.empty(dshape, dtype=float)

    # Compute each distance.
    for i in range(4):
        # Point indices for the other line.
        A = (i + 1) % 4
        B = (i + 3) % 4

        # Find the parameter t. Note we compute the dot product ourselves rather than
        # using np.dot() as the latter has different behaviours depending on the
        # dimensionality of the inputs.
        dirvec = P[B] - P[A]
        offvec = P[i] - P[A]
        t = np.clip(np.sum(dirvec * offvec, axis=-1) / np.sum(dirvec**2, axis=-1), 0, 1)

        # Calculate the distance between the starting point and the intermediate point.
        H = P[A] + t[..., np.newaxis] * dirvec
        d[..., i] = np.linalg.norm(P[i] - H, axis=-1)

    # Calculate the metrics assuming no intersections.
    dmin = d.min(axis=-1)
    dlpa = 0.5 * (d[..., :2].min(axis=-1) + d[..., 2:].min(axis=-1))

    # These may have collapsed to single values. This ensures they are arrays so we can
    # index them later.
    dmin = np.atleast_1d(dmin)
    dlpa = np.atleast_1d(dlpa)

    # Precompute the three differences needed for the determinants. The naming BA means
    # B - A and so on.
    BA = P[2] - P[0]
    CD = P[1] - P[3]
    CA = P[1] - P[0]

    # Compute the determinants.
    detA = BA[..., 0] * CD[..., 1] - CD[..., 0] * BA[..., 1]
    detAr = CA[..., 0] * CD[..., 1] - CD[..., 0] * CA[..., 1]
    detAs = BA[..., 0] * CA[..., 1] - CA[..., 0] * BA[..., 1]

    # Adjust the signs to simplify the comparison.
    sgn = np.sign(detA)
    detA = np.abs(detA)
    detAr *= sgn
    detAs *= sgn

    # Find which partner legs intersect the base leg.
    intersects = (
        ~np.isclose(detA, 0, rtol=0, atol=1e-9)
        & (0 <= detAr)
        & (detAr <= detA)
        & (0 <= detAs)
        & (detAs <= detA)
    )

    # And adjust the distances accordingly.
    dmin[intersects] = 0
    dlpa[intersects] *= 0.5

    if return_all:
        return dmin, dlpa, d, intersects
    return dmin, dlpa

We can check it with a single partner segment:

In [ ]:
line_piece_distances(
    [[0, 0], [100, 0]],
    [[2, -10], [98, -10]],
)

Or with multiple partner segments. The partner array can have as many dimensions as you want, provided the vertex coordinates are in the last two (which must both have a length of 2).

In [ ]:
line_piece_distances(
    [[0, 0], [100, 0]],
    [
        [[2, -10], [98, -10]],
        [[2, -10], [98, 10]],
        [[-8, 7], [77, 1]],
    ],
)

If you also want the intermediate distances and the results of the intersection test, pass `return_all=True` to the function.

In [ ]:
lpmin, lpavg, d, intersects = line_piece_distances(
    [[0, 0], [100, 0]],
    [
        [[2, -10], [98, -10]],
        [[2, -10], [98, 10]],
        [[-8, 7], [77, 1]],
    ],
    return_all=True,
)

In [ ]:
lpmin

In [ ]:
lpavg

In [ ]:
d

In [ ]:
intersects